In [1]:
import requests
from bs4 import BeautifulSoup
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from zipfile import ZipFile
%matplotlib inline

In [2]:
def get_kickstarter_df_2019():
    
    response = requests.get('https://webrobots.io/kickstarter-datasets/')
    
    try:
        while response.status_code != 200:
            response = requests.get('https://webrobots.io/kickstarter-datasets/')
    except:
        return None

    soup = BeautifulSoup(response.content, 'lxml')

    all_lis = soup.find('div', class_ = 'fusion-text').find_all('li')

    zips_2019 = [li.find_all('a')[-1].get('href') for li in all_lis if '2019' in li.find_all('a')[-1].get('href')]
    zips_2019 = zips_2019[:3]

    all_dfs = []
    for link in zips_2019:
        response_zip = requests.get(link)

        zip_file = ZipFile(io.BytesIO(response_zip.content), 'r') 
        dfs_ = [pd.read_csv(zip_file.open(text_file.filename))
               for text_file in zip_file.infolist()
               if text_file.filename.endswith('.csv')]

        all_dfs.extend(dfs_)

    df_ = pd.concat(all_dfs, sort=False)

    return df_


In [3]:
df = get_kickstarter_df_2019()

In [4]:
df.shape

(630308, 38)

In [5]:
## save df

# commenting this cell as data change with time
# df.to_csv('../data/kickstarter_sub.zip', compression='zip')